In [28]:
import queue_ as qu
import pandas as pd
import numpy as np
import matplotlib as plt
import random as rm


## Эксперименты с различными параметрами

#### В ходе экспериментов будем вычислять следующие метрики:

    1) Среднее время пребывания требования в системе
    2) Вероятность ожидания (требование пришло, а очередь непустая)
    3) Среднее количество заявок в очереди
    4) Вероятность простоя системы (оба сервера свободны и очередь пустая)



In [29]:
def count_metrics(size_metrics, u_metrics, lam_metrics):
    queue = qu.Queue(seed=rm.randint(0, 10 ** 5), lam=lam_metrics, u=u_metrics, size=size_metrics)
    metrics = pd.DataFrame(columns=['Average time in the system',
                    'Probability of waiting',
                    'Average count of customers in the system',
                    '------------------'])

    metrics['Average time in the system'] = [i[2] - i[0] for i in queue.client_schedule]
    client_was_waiting = []
    truhnie = 0
    for i in range(len(queue.client_schedule)):
        if queue.client_schedule[i][1]-queue.client_schedule[i][0]:
            truhnie += 1
        client_was_waiting.append(truhnie/(i+1))
    metrics['Probability of waiting'] = client_was_waiting
    
    metrics['Average count of customers in the system'] = [queue._a(i[0]) for i in queue.client_schedule]
    people_num = [queue._a(i[0]) for i in queue.client_schedule]
    metrics['Average count of customers in the system'] = [sum(people_num[:i+1])/(i+1) for i in range(len(people_num))]

    return metrics

In [30]:
def draw_metrics(param_name, size_metrics, u_metrics, lam_metrics):

    x = np.linspace(0, 9.9, 100)
    y = 0
    '''plt.figure(figsize=(15,15))
    plt.suptitle(f'Метрики в зависимости от параметра {param_name}')

    i = 0

    for metric_name, values in test_metrics.items():
        plt.subplot(321+i)   
        plt.plot(params, values, 'o-')
        plt.title(metrics_titles[i])
        plt.xlabel(param_name)
        plt.ylabel(metric_name)
        plt.tight_layout()
        plt.grid()
        i += 1

    plt.show()'''

In [ ]:
print(count_metrics())

# xtnj


In [33]:
x = np.linspace(0.1, 9.9, 99)
y = count_metrics(size_metrics=5, u_metrics=2, lam_metrics = x[0])
y = [y['Average time in the system']]
y



[0    0.298401
 1    0.814284
 2    0.219713
 3    1.690432
 Name: Average time in the system, dtype: float64]